In [364]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import class_weight
import tensorflow as tf

In [308]:
#Constantes
BATCH_SIZE = 7

# 1. Recolher dados

In [309]:
def get_data_from_mat(train_file,test_file):
    train_mat = scipy.io.loadmat(train_file) 
    test_mat = scipy.io.loadmat(test_file) 
    print(train_mat.keys())
    print(test_mat.keys())
    train_np = np.array(train_mat['train_data']).transpose(2,0,1)
    test_np = np.array(test_mat['test_data']).transpose(2,0,1)
    print('Train shape: ',train_np.shape)
    print('Test shape: ',test_np.shape)
    return  train_np, test_np


train_data, test_data = get_data_from_mat('train_data.mat','test_data.mat')

dict_keys(['__header__', '__version__', '__globals__', 'train_data'])
dict_keys(['__header__', '__version__', '__globals__', 'test_data'])
Train shape:  (112, 90, 90)
Test shape:  (28, 90, 90)


In [310]:
train_data[0]

array([[0.        , 0.        , 0.13152513, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.13152513, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.28060623],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.28060623, 0.        ,
        0.        ]])

# 2. Preparar os datasets de treino e de teste

In [311]:
#Tornar cada matriz num vetor de uma dimensão (passar de 90x90 para 8100) e armazenar todos os valores de cada matriz
#O dicionário utilizado armazena, todos os valores com indíce X, pertencentes às diferentes 112 matrizes, sendo esse índice
#a chave do dic.
#Por exemplo: dictAttributes[0] contém o primeiro elemento de cada vetor anteriormente criado
dictAttributes = dict()
for i in range(len(train_data)):# len(train_data) - 112
    flatten_data = train_data[i].flatten()
    for j in range(len(flatten_data)):
        dictAttributes.setdefault(j,list())
        dictAttributes[j].append(flatten_data[j])

#Tornar cada matriz num vetor de uma dimensão (passar de 90x90 para 8100) e armazenar todos os valores de cada matriz
#O dicionário utilizado armazena, todos os valores com indíce X, pertencentes às diferentes 112 matrizes, sendo esse índice
#a chave do dic.
#Por exemplo: dictAttributes[0] contém o primeiro elemento de cada vetor anteriormente criado
dictAttributesTest = dict()
for i in range(len(test_data)):# len(test_data) - 28
    flatten_data = test_data[i].flatten()
    for j in range(len(flatten_data)):
        dictAttributesTest.setdefault(j,list())
        dictAttributesTest[j].append(flatten_data[j])
        
#Nesta fase criam-se colunas com todos os elementos que estão numa dada posição do vetor, a label da coluna é essa posição
scans_series = []
for k in dictAttributes.keys():
    scans_series.append(pd.Series(dictAttributes.get(k)))
    
#Nesta fase criam-se colunas com todos os elementos que estão numa dada posição do vetor, a label da coluna é essa posição
scans_series_test = []
for k in dictAttributesTest.keys():
    scans_series_test.append(pd.Series(dictAttributesTest.get(k)))

In [312]:
train_data_series = pd.concat(scans_series,axis=1)
train_data_series

,0,1,2,3,4,5,6,7,8,9,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,0.0,0.0,0.131525,0.0,0.0,0.0,0.271538,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.585006,0.0,0.280606,0.0,0.0
1,0.0,0.0,0.117704,0.0,0.0,0.0,0.316890,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.599437,0.0,0.207478,0.0,0.0
2,0.0,0.0,0.155806,0.0,0.0,0.0,0.348072,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.630374,0.0,0.246472,0.0,0.0
3,0.0,0.0,0.145214,0.0,0.0,0.0,0.310270,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.562505,0.0,0.287888,0.0,0.0
4,0.0,0.0,0.161360,0.0,0.0,0.0,0.385560,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.564043,0.0,0.234284,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.0,0.0,0.215767,0.0,0.0,0.0,0.390332,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.011150,0.0,0.660292,0.0,0.302675,0.0,0.0
108,0.0,0.0,0.143005,0.0,0.0,0.0,0.306380,0.0,0.0,0.0,...,0.0,0.011171,0.0,0.019141,0.0,0.630415,0.0,0.204125,0.0,0.0
109,0.0,0.0,0.116085,0.0,0.0,0.0,0.290941,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.574355,0.0,0.246784,0.0,0.0
110,0.0,0.0,0.156654,0.0,0.0,0.0,0.357540,0.0,0.0,0.0,...,0.0,0.010225,0.0,0.000000,0.0,0.631057,0.0,0.216259,0.0,0.0


In [313]:
test_data_series = pd.concat(scans_series_test,axis=1)
test_data_series

,0,1,2,3,4,5,6,7,8,9,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,0.0,0.0,0.140891,0.000000,0.0,0.0,0.332442,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.612109,0.0,0.193069,0.0,0.0
1,0.0,0.0,0.147297,0.000000,0.0,0.0,0.326313,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.635466,0.0,0.275949,0.0,0.0
2,0.0,0.0,0.091724,0.000000,0.0,0.0,0.338758,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.577096,0.0,0.260915,0.0,0.0
3,0.0,0.0,0.095972,0.000000,0.0,0.0,0.308292,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.535117,0.0,0.198058,0.0,0.0
4,0.0,0.0,0.129573,0.000000,0.0,0.0,0.337631,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.590000,0.0,0.266638,0.0,0.0
5,0.0,0.0,0.142985,0.000000,0.0,0.0,0.350861,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.702736,0.0,0.267522,0.0,0.0
6,0.0,0.0,0.131228,0.000000,0.0,0.0,0.309908,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.612072,0.0,0.282661,0.0,0.0
7,0.0,0.0,0.101044,0.000000,0.0,0.0,0.337020,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.014537,0.0,0.611550,0.0,0.291660,0.0,0.0
8,0.0,0.0,0.230942,0.000000,0.0,0.0,0.409840,0.0,0.0,0.0,...,0.0,0.011865,0.0,0.000000,0.0,0.608007,0.0,0.278882,0.0,0.0
9,0.0,0.0,0.134479,0.000000,0.0,0.0,0.278180,0.0,0.0,0.0,...,0.0,0.024165,0.0,0.000000,0.0,0.633834,0.0,0.284398,0.0,0.0


## 2.1. Adicionar mais informação da literacia

In [314]:
TRAIN_CSV_PATH = 'train.csv'
TEST_CSV_PATH = 'test.csv'
rest_train_df = pd.read_csv(TRAIN_CSV_PATH)
rest_train_df.head(10) #Tem 112 linhas

,id,age,sex,education
0,1,13,1,7
1,2,14,0,8
2,3,15,1,9
3,4,15,1,9
4,5,15,1,9
5,6,15,1,9
6,7,16,0,10
7,8,16,1,10
8,9,16,0,10
9,10,17,0,11


In [315]:
rest_test_df = pd.read_csv(TEST_CSV_PATH)
rest_test_df.head(10) #Tem 28 linhas

,id,sex,education
0,1,0,13
1,2,0,11
2,3,1,9
3,4,1,13
4,5,0,12
5,6,0,17
6,7,0,9
7,8,0,4
8,9,1,9
9,10,1,4


In [316]:
def normalize(X):
    Xmax = np.max(X, axis = 0)
    Xmin = np.min(X, axis = 0)
    return (X-Xmin)/(Xmax-Xmin)

targets_train = rest_train_df['age']
education_train = normalize(rest_train_df['education'])
sex_train = rest_train_df['sex'].astype('float32')
education_test = normalize(rest_test_df['education'])
sex_test = rest_test_df['sex'].astype('float32')

In [317]:
#Juntar o resto à conectividade
train_data_series = pd.concat([targets_train, sex_train, education_train] + scans_series,axis=1)
train_data_series

,age,sex,education,0,1,2,3,4,5,6,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,13,1.0,0.35,0.0,0.0,0.131525,0.0,0.0,0.0,0.271538,...,0.0,0.000000,0.0,0.000000,0.0,0.585006,0.0,0.280606,0.0,0.0
1,14,0.0,0.40,0.0,0.0,0.117704,0.0,0.0,0.0,0.316890,...,0.0,0.000000,0.0,0.000000,0.0,0.599437,0.0,0.207478,0.0,0.0
2,15,1.0,0.45,0.0,0.0,0.155806,0.0,0.0,0.0,0.348072,...,0.0,0.000000,0.0,0.000000,0.0,0.630374,0.0,0.246472,0.0,0.0
3,15,1.0,0.45,0.0,0.0,0.145214,0.0,0.0,0.0,0.310270,...,0.0,0.000000,0.0,0.000000,0.0,0.562505,0.0,0.287888,0.0,0.0
4,15,1.0,0.45,0.0,0.0,0.161360,0.0,0.0,0.0,0.385560,...,0.0,0.000000,0.0,0.000000,0.0,0.564043,0.0,0.234284,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,77,1.0,0.20,0.0,0.0,0.215767,0.0,0.0,0.0,0.390332,...,0.0,0.000000,0.0,0.011150,0.0,0.660292,0.0,0.302675,0.0,0.0
108,67,0.0,0.20,0.0,0.0,0.143005,0.0,0.0,0.0,0.306380,...,0.0,0.011171,0.0,0.019141,0.0,0.630415,0.0,0.204125,0.0,0.0
109,55,0.0,0.20,0.0,0.0,0.116085,0.0,0.0,0.0,0.290941,...,0.0,0.000000,0.0,0.000000,0.0,0.574355,0.0,0.246784,0.0,0.0
110,76,1.0,0.15,0.0,0.0,0.156654,0.0,0.0,0.0,0.357540,...,0.0,0.010225,0.0,0.000000,0.0,0.631057,0.0,0.216259,0.0,0.0


In [318]:
train_df_all = pd.DataFrame(train_data_series)
train_df_all

,age,sex,education,0,1,2,3,4,5,6,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,13,1.0,0.35,0.0,0.0,0.131525,0.0,0.0,0.0,0.271538,...,0.0,0.000000,0.0,0.000000,0.0,0.585006,0.0,0.280606,0.0,0.0
1,14,0.0,0.40,0.0,0.0,0.117704,0.0,0.0,0.0,0.316890,...,0.0,0.000000,0.0,0.000000,0.0,0.599437,0.0,0.207478,0.0,0.0
2,15,1.0,0.45,0.0,0.0,0.155806,0.0,0.0,0.0,0.348072,...,0.0,0.000000,0.0,0.000000,0.0,0.630374,0.0,0.246472,0.0,0.0
3,15,1.0,0.45,0.0,0.0,0.145214,0.0,0.0,0.0,0.310270,...,0.0,0.000000,0.0,0.000000,0.0,0.562505,0.0,0.287888,0.0,0.0
4,15,1.0,0.45,0.0,0.0,0.161360,0.0,0.0,0.0,0.385560,...,0.0,0.000000,0.0,0.000000,0.0,0.564043,0.0,0.234284,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,77,1.0,0.20,0.0,0.0,0.215767,0.0,0.0,0.0,0.390332,...,0.0,0.000000,0.0,0.011150,0.0,0.660292,0.0,0.302675,0.0,0.0
108,67,0.0,0.20,0.0,0.0,0.143005,0.0,0.0,0.0,0.306380,...,0.0,0.011171,0.0,0.019141,0.0,0.630415,0.0,0.204125,0.0,0.0
109,55,0.0,0.20,0.0,0.0,0.116085,0.0,0.0,0.0,0.290941,...,0.0,0.000000,0.0,0.000000,0.0,0.574355,0.0,0.246784,0.0,0.0
110,76,1.0,0.15,0.0,0.0,0.156654,0.0,0.0,0.0,0.357540,...,0.0,0.010225,0.0,0.000000,0.0,0.631057,0.0,0.216259,0.0,0.0


In [319]:
#Juntar o resto à conectividade
test_df_all = pd.DataFrame(pd.concat([sex_test, education_test] + scans_series_test,axis=1))
test_df_all

,sex,education,0,1,2,3,4,5,6,7,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,0.0,0.764706,0.0,0.0,0.140891,0.000000,0.0,0.0,0.332442,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.612109,0.0,0.193069,0.0,0.0
1,0.0,0.647059,0.0,0.0,0.147297,0.000000,0.0,0.0,0.326313,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.635466,0.0,0.275949,0.0,0.0
2,1.0,0.529412,0.0,0.0,0.091724,0.000000,0.0,0.0,0.338758,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.577096,0.0,0.260915,0.0,0.0
3,1.0,0.764706,0.0,0.0,0.095972,0.000000,0.0,0.0,0.308292,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.535117,0.0,0.198058,0.0,0.0
4,0.0,0.705882,0.0,0.0,0.129573,0.000000,0.0,0.0,0.337631,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.590000,0.0,0.266638,0.0,0.0
5,0.0,1.000000,0.0,0.0,0.142985,0.000000,0.0,0.0,0.350861,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.702736,0.0,0.267522,0.0,0.0
6,0.0,0.529412,0.0,0.0,0.131228,0.000000,0.0,0.0,0.309908,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.612072,0.0,0.282661,0.0,0.0
7,0.0,0.235294,0.0,0.0,0.101044,0.000000,0.0,0.0,0.337020,0.0,...,0.0,0.000000,0.0,0.014537,0.0,0.611550,0.0,0.291660,0.0,0.0
8,1.0,0.529412,0.0,0.0,0.230942,0.000000,0.0,0.0,0.409840,0.0,...,0.0,0.011865,0.0,0.000000,0.0,0.608007,0.0,0.278882,0.0,0.0
9,1.0,0.235294,0.0,0.0,0.134479,0.000000,0.0,0.0,0.278180,0.0,...,0.0,0.024165,0.0,0.000000,0.0,0.633834,0.0,0.284398,0.0,0.0


In [354]:
x_train = np.expand_dims(train_df_all.values[:,1:],axis=2)
x_test = np.expand_dims(test_df_all.values[:,:],axis=2)
x_train.shape

(112, 8102, 1)

In [342]:
#Estratégia para alteração dos pesos de modo a não prejudicar as classes que aparecem menos
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(train_df_all.values[:,0].astype(int))
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [340]:
#y_train = np.expand_dims(train_df_all.values[:,0], axis=1).astype(int)
#y_train.shape

(112, 1)

In [353]:
inputs = tf.keras.Input(shape=(8102,1))
x = tf.keras.layers.Conv1D(filters=512, kernel_size=3, activation='relu')(inputs)
x = tf.keras.layers.MaxPool1D()(x)

x = tf.keras.layers.Conv1D(filters=256, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPool1D()(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse',
    metrics =  ['mae']
)

history = model.fit(
    x_train,
    y_train,
    epochs=25,
    batch_size=8,
    validation_split=0.1,
    class_weight=class_weight_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/25
13/13 [==============================] - 18s 1s/step - loss: 625.1148 - mae: 18.9061 - val_loss: 771.3630 - val_mae: 26.6830
Epoch 2/25
13/13 [==============================] - 16s 1s/step - loss: 274.9019 - mae: 14.8733 - val_loss: 649.4863 - val_mae: 24.2945
Epoch 3/25
13/13 [==============================] - 16s 1s/step - loss: 237.7596 - mae: 13.0894 - val_loss: 171.1865 - val_mae: 11.4299
Epoch 4/25
13/13 [==============================] - 17s 1s/step - loss: 208.0418 - mae: 13.7096 - val_loss: 552.8007 - val_mae: 22.2227
Epoch 5/25
13/13 [==============================] - 17s 1s/step - loss: 213.6623 - mae: 12.9942 - val_loss: 186.3422 - val_mae: 11.9345
Epoch 6/25
13/13 [==============================] - 17s 1s/step - loss: 166.4021 - mae: 12.3203 - val_loss: 319.4128 - val_mae: 16.1285
Epoch 7/25
13/13 [==============================] - 17s 1s/step - loss: 165.8604 - mae: 12.2849 - val_loss: 171.0578 - val_mae: 11.5340
Epoch 8/25
13/13 [==============================

In [298]:
model.summary()

Model: "model_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_84 (InputLayer)       [(None, 8102, 1)]         0         
                                                                 
 conv1d_123 (Conv1D)         (None, 8100, 1024)        4096      
                                                                 
 max_pooling1d_108 (MaxPooli  (None, 4050, 1024)       0         
 ng1D)                                                           
                                                                 
 conv1d_124 (Conv1D)         (None, 4048, 512)         1573376   
                                                                 
 max_pooling1d_109 (MaxPooli  (None, 2024, 512)        0         
 ng1D)                                                           
                                                                 
 global_average_pooling1d_45  (None, 512)              0  

In [365]:
preds = model.predict(x_test)
preds = [int(age) for age in preds]
preds = le.inverse_transform(preds)

with open('test_submission.csv', 'w', encoding='UTF8', newline='') as f:
    count_id = 1
    writer = csv.writer(f)
    writer.writerow(['id','age'])
    for age in preds:
        line = [count_id, age]
        writer.writerow(line)
        count_id += 1